In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
import numpy as np

df = pd.read_parquet('./data')
print(f'total dataset: {len(df)} rows')

## filters
# df = df[df['stabilized_spread'] < 10]  # nem tul nagy spread
df = df[df['stabilized_spread'] > 0.3]  # nem tul kis spread

for col in df.columns:
    if 'nr_trades' in col or 'price_delta' in col and not col.startswith('last'):
        df.drop(col, axis=1, inplace=True)
    # if 'nr_trades' in col:
    #     df.drop(col, axis=1, inplace=True)
    if 'past' in col:
        df.drop(col, axis=1, inplace=True)
    if '_spread' == col[1:]:
        df.drop(col, axis=1, inplace=True)
    pass

df_min = df[df.wave_direction == 'min'].copy()
df_max = df[df.wave_direction == 'max'].copy()

df_min.drop('wave_direction', axis=1, inplace=True)
df_max.drop('wave_direction', axis=1, inplace=True)
df = None
print(f'min dataset: {len(df_min)}')
print(f'max dataset: {len(df_max)}')

df_min.sort_index(axis=1, inplace=True)
df_max.sort_index(axis=1, inplace=True)

display(df_max)

# df_max.last_price_delta_since_stabilized = df_max.last_price_delta_since_stabilized * -1
df_max.last_price_delta_since_stabilized.describe()

total dataset: 186831 rows
min dataset: 63666
max dataset: 64555


,0_amount_mean,1_amount_mean,2_amount_mean,3_amount_mean,4_amount_mean,last_price_delta_since_stabilized,stabilized_amount_mean,stabilized_at_ms,stabilized_gasp,stabilized_spread
index,,,,,,,,,,
0,0.0305,0.0815,0.0792,0.0767,0.0725,-0.78,0.0706,108,-0.4798,0.91
0,0.0078,0.0072,0.0122,0.0173,0.0301,-1.84,0.0295,186,0.2329,1.67
0,0.0348,0.0531,0.0503,0.0489,0.0481,-0.74,0.0469,151,-0.8172,1.83
0,0.0031,0.0027,0.0036,0.0058,0.0188,-0.89,0.0167,170,-0.9874,1.81
0,0.0318,0.0315,0.0317,0.0341,0.0360,0.00,0.0345,822,-0.5774,3.55
...,...,...,...,...,...,...,...,...,...,...
0,0.0034,0.0036,0.0030,0.0025,0.0317,0.00,0.0297,486,-0.6513,1.13
0,0.0135,0.0127,0.0138,0.0132,0.0125,0.00,0.0115,630,-0.3390,0.62
0,0.4005,0.3722,0.3474,0.3068,0.1671,0.00,0.1633,857,-0.6240,0.94


count    64555.000000
mean        -0.577874
std          0.988378
min        -17.160000
25%         -0.730000
50%         -0.200000
75%          0.000000
max          0.000000
Name: last_price_delta_since_stabilized, dtype: float64

In [9]:
a = df_min.corr().last_price_delta_since_stabilized * 1000
display(a.sort_values(axis=0, ascending=False))

a = df_max.corr().last_price_delta_since_stabilized * 1000
display(a.sort_values(axis=0, ascending=False))


last_price_delta_since_stabilized    1000.000000
stabilized_spread                     125.431121
stabilized_amount_mean                 67.120783
4_amount_mean                          58.007518
3_amount_mean                          55.873468
2_amount_mean                          51.272224
1_amount_mean                          49.941988
0_amount_mean                          48.579444
stabilized_gasp                        14.430473
stabilized_at_ms                     -298.990994
Name: last_price_delta_since_stabilized, dtype: float64

last_price_delta_since_stabilized    1000.000000
stabilized_at_ms                      293.916880
stabilized_gasp                         5.985565
1_amount_mean                         -41.694275
0_amount_mean                         -41.802218
2_amount_mean                         -41.917761
3_amount_mean                         -49.123865
4_amount_mean                         -53.026412
stabilized_amount_mean                -64.591582
stabilized_spread                    -122.949924
Name: last_price_delta_since_stabilized, dtype: float64

In [11]:
from spreadsurfer.price_engine import FeatureEngineer
from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler


def train(df, loss_function):
    # model = CatBoostRegressor(learning_rate=0.01, depth=7, loss_function=loss_function, random_state=0, verbose=False, iterations=1500)
    model = CatBoostRegressor(learning_rate=0.15, depth=6, loss_function=loss_function, random_state=0, verbose=False)
    pipeline = Pipeline(steps=[
        ('preprocessor', FeatureEngineer()),
        ('model', model)
    ])

    df = df.copy()
    y = df['last_price_delta_since_stabilized']
    X = df.drop('last_price_delta_since_stabilized', axis=1)
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.7, test_size=0.3, random_state=0)

    pipeline.fit(X_train, y_train)
    score = pipeline.score(X_valid, y_valid)
    display(score)
    return model, score


min_model, min_score = train(df_min, loss_function='MAPE')
max_model, max_score = train(df_max, loss_function='MAPE')
print('--------')
# min_model, min_score = train(df_min, loss_function='RMSE')
# max_model, max_score = train(df_max, loss_function='RMSE')

# min_model, min_score = train(df_min, loss_function='Poisson') # 0.12
# max_model, max_score = train(df_max, loss_function='Poisson') # 0.14

# min_model, min_score = train(df_min, loss_function='Huber:delta=0.4')
# max_model, max_score = train(df_max, loss_function='Huber:delta=0.4')

# max_model, max_score = train(df_max, loss_function='Quantile:alpha=0.8')
# max_model, max_score = train(df_max, loss_function='Quantile:alpha=0.7')
# max_model, max_score = train(df_max, loss_function='Quantile:alpha=0.6')
# max_model, max_score = train(df_max, loss_function='Quantile:alpha=0.5')
# max_model, max_score = train(df_max, loss_function='Quantile:alpha=0.4')
# max_model, max_score = train(df_max, loss_function='Quantile:alpha=0.3')
# max_model, max_score = train(df_max, loss_function='Quantile:alpha=0.2')

# max_model, max_score = train(df_max, loss_function='Quantile:alpha=0.7')



-0.008416009133632274

-0.026631889282630272

--------


In [12]:
from spreadsurfer import now_isoformat

save = True
if save:
    min_model.save_model(f'./models/{now_isoformat()}-min_-mape-score-{1000 * round(min_score, 3)}.cat')
    max_model.save_model(f'./models/{now_isoformat()}-max_mape-score-{1000 * round(max_score, 3)}.cat')
